In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd


class MicrogridEnv(gym.Env):

    def __init__(self, data):
        super(MicrogridEnv, self).__init__()

        self.data = data
        self.action_space = spaces.Box(low=0.0, high=1.0, shape=(1,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(4,), dtype=np.float32)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)

        self.current_step = np.random.randint(0, len(self.data))
        row = self.data.iloc[self.current_step]
        self.temp = row['Avg_Temperature_C']
        self.consumption = row['Energy_Consumption_kWh']
        self.household_size = row['Household_Size']
        self.peak_usage = row['Peak_Hours_Usage_kWh']

        self.grid_price = 0.12 + (0.10 * self.peak_usage)
        observation = np.array([self.temp, self.consumption, self.household_size, self.grid_price], dtype=np.float32)
        
        return observation, {}


    def step(self, action):
        shed_factor = action[0]

        cost = self.consumption * (1 - shed_factor) * self.grid_price
        discomfort = self.consumption * shed_factor * (0.5 * self.temp)

        reward = -cost - discomfort
        
        return np.array([0,0,0,0], dtype=np.float32), reward, False, False, {}